In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import fmin_l_bfgs_b

sns.set_theme(style="whitegrid")

In [3]:
class ANNRegression():
    # interno so posamezni primeri stolpci v matriki X, zunanje dosegljive funkcije pa obratno (kot zahtevano)
    def __init__(self, units=[], lambda_=0):
        self.units = units
        self.lambda_ = lambda_
        self.w = []


    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_prime(self, a):
        return a * (1 - a)
    

    # ti funkciji bosta drugacni za klasifikacijo
    def last_layer(self, x):
        return x
    def preprocess_y(self, y):
        if len(y.shape) == 1:
            y = y.reshape(-1, 1)
        return y
    

    def params_to_weights(self, params):
        return [params[self.params_upto_l[l]:self.params_upto_l[l+1]].reshape(self.w[l].shape) for l in range(len(self.w))]
    
    def weights_to_params(self, weights):
        return np.concatenate([w.flatten() for w in weights])
    

    def initialize(self, X, y, scale_weights=1):
        y = self.preprocess_y(y)
        n = X.shape[0]

        # velikosti plasti
        sizes = [X.shape[1]] + self.units + [y.shape[1]]
        self.sizes = sizes

        # incializira utezi in ustvari matrike gradientov
        self.w = [np.random.normal(0, scale_weights / np.sqrt(sizes[i]), (sizes[i+1], sizes[i]+1)) for i in range(len(sizes)-1)]
        self.dw = [np.zeros_like(w) for w in self.w]

        # za pretvorbo vektorja parametrov v matrike
        self.params_upto_l = [0] + [np.sum([np.prod(w.shape) for w in self.w[:l]]) for l in range(1, len(self.w)+1)]

        # ustvari matrike aktivacij
        self.a = [np.ones((sizes[l]+1, n)) for l in range(len(sizes))]
        self.a[-1] = np.ones((sizes[-1], n))
        self.delta = [np.zeros_like(a) for a in self.a]

        self.n_weights = np.sum([np.prod(w[:, :-1].shape) for w in self.w])

        return X, y
    
    
    def fit(self, X, y):
        X, y = self.initialize(X, y)

        params = self.weights_to_params(self.w)
        params_opt, _, _ = fmin_l_bfgs_b(lambda ps, X, y: self.cost_and_grad(X, y, self.params_to_weights(ps)), params, args=(X.T, y.T))
        # params_opt = params
        self.w = self.params_to_weights(params_opt)

        return self
    

    def forward_pass(self, X, ws):
        a = self.a
        a[0][:-1, :] = X
        for l in range(1, len(self.w)):
            a[l][:-1, :] = self.sigmoid(ws[l-1] @ a[l-1])

        l = len(self.w)
        a[l] = self.last_layer(ws[l-1] @ a[l-1])
        return a
    

    def cost_function(self, preds, y):
        return 0.5 * np.sum((preds - y)**2) / y.shape[1]

    def cost(self, y, aa, ws):
        # izvzamemo zadnji stolpec utezi, ki predstavlja bias
        return self.cost_function(aa[-1], y) + self.lambda_ / (2 * self.n_weights) * np.sum([np.sum(w[:, :-1]**2) for w in ws])
    

    
    def last_layer_derivative(self, a, y):
        return a - y


    def cost_and_grad(self, X, y, ws=None):
        if ws is None:
            ws = self.w
        dw = self.dw

        n = X.shape[1]

        # forward pass
        aa = self.forward_pass(X, ws)
        cost = self.cost(y, aa, ws)
        # zadnji layer
        delta = self.last_layer_derivative(aa[-1], y)
        # backward pass
        for l in range(len(ws)-1, -1, -1):
            dw[l] = delta @ aa[l].T / n
            dw[l][:, :-1] += self.lambda_ * ws[l][:, :-1] / self.n_weights
            delta = (ws[l][:, :-1].T @ delta) * self.sigmoid_prime(aa[l][:-1, :])

        return cost, self.weights_to_params(dw)


    def predict(self, X):
        # ustvari matrike aktivacij
        n = X.shape[0]
        self.a = [np.ones((self.sizes[l]+1, n)) for l in range(len(self.sizes))]
        self.a[-1] = np.ones((self.sizes[-1], n))

        preds = self.forward_pass(X.T, self.w)[-1]
        if preds.shape[0] == 1:
            return preds.flatten()
        return preds.T

    def weights(self):
        return [w.T for w in self.w]
    
    def cost_at_params(self, params, X, y):
        X = X.T
        y = self.preprocess_y(y).T
        ws = self.params_to_weights(params)
        aa = self.forward_pass(X, ws)
        return self.cost(y, aa, ws)


class ANNClassification(ANNRegression):
    def last_layer(self, x):
        # softmax
        x = x - np.mean(x, axis=0, keepdims=True)
        u = np.exp(x)
        return u / np.sum(u, axis=0, keepdims=True)

    def preprocess_y(self, y):
        y = super().preprocess_y(y)
        # one hot encoding for final layer
        y = (y == np.unique(y)).astype(int)
        return y

    def cost_function(self, preds, y):
        # cross entropy
        return -np.sum(y * np.log(preds)) / y.shape[1]

In [4]:
X = np.array([[0, 0],
                    [0, 1],
                    [1, 0],
                    [1, 1]])
y = np.array([0, 1, 2, 3])
hard_y = np.array([0, 1, 1, 0])

In [5]:
y = hard_y
# y = np.array([0, 1, 1, 1])
fitter = ANNClassification(units=[3], lambda_=0.001)
m = fitter.fit(X, y)
# fitter.initialize(X, y)
# m = fitter

In [8]:
np.random.seed(42)
fitter = ANNClassification(units=[3], lambda_=0.001)
fitter.initialize(X, hard_y)
compare_numerical_gradient(fitter, X, hard_y)
m = fitter.fit(X, hard_y)
compare_numerical_gradient(m, X, hard_y)

Cost: 1.1587204138
Mean relative difference between numerical and analytical gradient:  7.2e-09
Max relative difference between numerical and analytical gradient:  5.89e-08
Stanard deviation of relative difference between numerical and analytical gradient:  1.73e-08
Cost: 0.0154612865
Mean relative difference between numerical and analytical gradient:  1.06812e-05
Max relative difference between numerical and analytical gradient:  7.24198e-05
Stanard deviation of relative difference between numerical and analytical gradient:  1.66335e-05


In [9]:
np.random.seed(42)
fitter = ANNRegression(units=[3], lambda_=0.001)
fitter.initialize(X, y)
compare_numerical_gradient(fitter, X, y)
m = fitter.fit(X, y)
compare_numerical_gradient(m, X, y)

Cost: 0.240934776
Mean relative difference between numerical and analytical gradient:  4e-10
Max relative difference between numerical and analytical gradient:  1.2e-09
Stanard deviation of relative difference between numerical and analytical gradient:  3e-10
Cost: 0.0020499033
Mean relative difference between numerical and analytical gradient:  6.657e-07
Max relative difference between numerical and analytical gradient:  8.0686e-06
Stanard deviation of relative difference between numerical and analytical gradient:  2.1375e-06


In [2609]:
pred = m.predict(X)

In [2610]:
pred.round(5)

array([[0.99741, 0.00259],
       [0.003  , 0.997  ],
       [0.00217, 0.99783],
       [0.99741, 0.00259]])

In [2611]:
yy = m.preprocess_y(y)
-np.sum(yy * np.log(pred)) / y.shape[0]

0.002587959758798886

In [2638]:
cost, grad = m.cost_and_grad(X.T, m.preprocess_y(y).T)

In [7]:
def compare_numerical_gradient(model, X, y, eps=1e-6, precision=10):
    cost, grad = model.cost_and_grad(X.T, model.preprocess_y(y).T)
    params = model.weights_to_params(model.w)
    errors = np.zeros_like(params)
    estimates = np.zeros_like(params)
    for i in range(len(params)):
        params[i] += eps
        c1 = model.cost_at_params(params, X, y)
        params[i] -= 2*eps
        c2 = model.cost_at_params(params, X, y)
        params[i] += eps
        estimate = (c1 - c2) / (2*eps)
        estimates[i] = estimate
        errors[i] = 2 * np.abs(estimate - grad[i]) / np.abs(estimate + grad[i])

    print("Cost:", cost.round(10))
    print("Mean relative difference between numerical and analytical gradient: ", np.mean(errors).round(precision))
    print("Max relative difference between numerical and analytical gradient: ", np.max(errors).round(precision))
    print("Stanard deviation of relative difference between numerical and analytical gradient: ", np.std(errors).round(precision))

In [2614]:
compare_numerical_gradient(m, X, y)

Cost: 0.0154612865
Mean relative difference between numerical and analytical gradient:  1.06812e-05
Max relative difference between numerical and analytical gradient:  7.24198e-05
Stanard deviation of relative difference between numerical and analytical gradient:  1.66335e-05


---

In [2639]:
housing2r = pd.read_csv('housing2r.csv')

In [2640]:
housing2r = pd.read_csv('housing2r.csv')
housing2r_y = housing2r.y.to_numpy()
housing2r_X = housing2r.drop(columns=['y']).to_numpy()

In [2490]:
fitter = ANNRegression(units=[5], lambda_=0.1)
m = fitter.fit(housing2r_X, housing2r_y)
preds = m.predict(housing2r_X)

def mse(y, preds):
    return np.mean((y - preds)**2)

mse(housing2r_y, preds)

C:\Users\timkm\AppData\Local\Temp\ipykernel_20284\2989588649.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


23.423880257476753

In [2422]:
# import linear_regression as lr
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(housing2r_X, housing2r_y)
preds = lr.predict(housing2r_X)

mse(housing2r_y, preds)

34.01611489103591

In [2643]:
y = housing2r_y
x = housing2r_X
k = 4

n_poskusov = 3

np.random.seed(42)
idx = np.random.permutation(len(y))
x, y = x[idx], y[idx]
id_splits = np.array_split(idx, k)
x_folds, y_folds = np.array_split(x, k), np.array_split(y, k)
losses = [np.zeros(len(y)) for i in range(n_poskusov)]
preds = [np.zeros(len(y)) for i in range(n_poskusov)]
for i in range(k):
    x_tr = np.concatenate([x_folds[j] for j in range(k) if j != i])
    y_tr = np.concatenate([y_folds[j] for j in range(k) if j != i])
    x_test = x_folds[i]
    y_test = y_folds[i]


    lr = LinearRegression()
    lr.fit(x_tr, y_tr)
    p = lr.predict(x_test)

    losses[0][id_splits[i]] = mse(y_test, p)
    preds[0][id_splits[i]] = p

    fitter = ANNRegression(units=[], lambda_=0.0)
    m = fitter.fit(x_tr, y_tr)
    p = m.predict(x_test)
    losses[1][id_splits[i]] = mse(y_test, p)
    preds[1][id_splits[i]] = p

    fitter = ANNRegression(units=[5], lambda_=0.1)
    m = fitter.fit(x_tr, y_tr)
    p = m.predict(x_test)
    losses[2][id_splits[i]] = mse(y_test, p)
    preds[2][id_splits[i]] = p

C:\Users\timkm\AppData\Local\Temp\ipykernel_20284\3331413488.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


C:\Users\timkm\AppData\Local\Temp\ipykernel_20284\3331413488.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\timkm\AppData\Local\Temp\ipykernel_20284\3331413488.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\timkm\AppData\Local\Temp\ipykernel_20284\3331413488.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [2645]:
imena = ["LogisticRegression", "ANN no hidden layers", "ANN hidden layer"]
for i in range(n_poskusov):
    print("")
    print(imena[i])
    print(f"Loss {i}: {losses[i].mean()} +/- {losses[i].std() / np.sqrt(len(losses[i]))}")


LogisticRegression
Loss 0: 42.03348383433988 +/- 1.457389798447459

ANN no hidden layers
Loss 1: 42.04763266732574 +/- 1.4571986133446344

ANN hidden layer
Loss 2: 29.1414232103974 +/- 0.827573393093739


---

In [3]:
housing3 = pd.read_csv('housing3.csv')
housing3_y = housing3.Class
# encode categories as integers
housing3.Class = pd.Categorical(housing3.Class)
housing3.Class = housing3.Class.cat.codes
housing3_y = housing3.Class.to_numpy()

housing3_X = housing3.drop(columns=['Class']).to_numpy()
# housing3_y

In [4]:
from oldmodels import MultinomialLogReg

logreg = MultinomialLogReg()
model = logreg.build(housing3_X, housing3_y)
preds = model.predict(housing3_X)

def log_loss(y, p):
    return -np.log(p[np.arange(len(y)), y])

l1 = log_loss(housing3_y, preds)
l1.mean()

0.2678180189024065

In [2649]:
# accuracy
np.mean(housing3_y == np.argmax(preds, axis=1))

0.886

In [2650]:
fitter = ANNClassification(units=[3], lambda_=0.1)
m = fitter.fit(housing3_X, housing3_y)
preds = m.predict(housing3_X)

l2 = log_loss(housing3_y, preds)
l2.mean()

C:\Users\timkm\AppData\Local\Temp\ipykernel_20284\3331413488.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


0.2298319324030689

In [2651]:
np.mean(housing3_y == np.argmax(preds, axis=1))

0.898

In [2660]:
y = housing3_y
x = housing3_X
k = 4

n_poskusov = 2

np.random.seed(42)
idx = np.random.permutation(len(y))
x, y = x[idx], y[idx]
id_splits = np.array_split(idx, k)
x_folds, y_folds = np.array_split(x, k), np.array_split(y, k)
losses = [np.zeros(len(y)) for i in range(n_poskusov)]
accuracies = [np.zeros(len(y)) for i in range(n_poskusov)]
preds = [np.zeros((len(y), 2)) for i in range(n_poskusov)]
for i in range(k):
    x_tr = np.concatenate([x_folds[j] for j in range(k) if j != i])
    y_tr = np.concatenate([y_folds[j] for j in range(k) if j != i])
    x_test = x_folds[i]
    y_test = y_folds[i]

    model = logreg.build(x_tr, y_tr)
    p = model.predict(x_test)
    losses[0][id_splits[i]] = log_loss(y_test, p)
    accuracies[0][id_splits[i]] = (y_test == np.argmax(p, axis=1))
    preds[0][id_splits[i], :] = p

    model = ANNClassification(units=[3], lambda_=0.5)
    model.fit(x_tr, y_tr)
    p = model.predict(x_test)
    losses[1][id_splits[i]] = log_loss(y_test, p)
    accuracies[1][id_splits[i]] = (y_test == np.argmax(p, axis=1))
    preds[1][id_splits[i], :] = p

C:\Users\timkm\AppData\Local\Temp\ipykernel_20284\834874647.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [2661]:
imena = ["MultinomialRegression", "ANN 1 hidden layer"]
for i in range(n_poskusov):
    print("")
    print(imena[i])
    print(f"Loss {i}: {losses[i].mean()} +/- {losses[i].std() / np.sqrt(len(losses[i]))}")
    print(f"Accuracy {i}: {accuracies[i].mean()} +/- {accuracies[i].std() / np.sqrt(len(accuracies[i]))}")


MultinomialRegression
Loss 0: 0.32145574599727444 +/- 0.032664724150925965
Accuracy 0: 0.872 +/- 0.0149409504383088

ANN 1 hidden layer
Loss 1: 0.2968973632275739 +/- 0.022186539798198898
Accuracy 1: 0.864 +/- 0.01532997064576446


----

In [51]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

train_X = df_train.drop(columns=['id', 'target']).to_numpy(dtype=np.float32)
train_X.shape

(50000, 93)

In [52]:
df_train.target = df_train.target.map(lambda s: int(s[-1]) - 1)

In [53]:
train_y = df_train.target.to_numpy()
train_y.shape

(50000,)

In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# scaler = StandardScaler()
# scaler = scaler.fit(train_X)
# train_X = scaler.transform(train_X)

In [55]:
def log_loss(y, p):
    return -np.log(p[np.arange(len(y)), y])

In [18]:
# import multinomial logistic regression from sklearn



logreg = LogisticRegression(multi_class='multinomial', max_iter=1000)
logreg.fit(train_X, train_y)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [19]:
preds = logreg.predict(train_X)
print(np.mean(train_y == preds))

from sklearn.metrics import log_loss as sk_log_loss

preds = logreg.predict_proba(train_X)
print(sk_log_loss(train_y, preds))
print(log_loss(train_y, preds).mean())


0.7673
0.622205378313374
0.6222053783133737


In [15]:
y = train_y
x = train_X
k = 4
n_poskusov = 2

np.random.seed(42)

idx = np.random.permutation(len(y))
x, y = x[idx], y[idx]


id_splits = np.array_split(idx, k)
x_folds, y_folds = np.array_split(x, k), np.array_split(y, k)

losses = [np.zeros(len(y)) for i in range(n_poskusov)]
accuracies = [np.zeros(len(y)) for i in range(n_poskusov)]
preds = [np.zeros((len(y), 9)) for i in range(n_poskusov)]

models = []

for i in range(k):
    x_tr = np.concatenate([x_folds[j] for j in range(k) if j != i])
    y_tr = np.concatenate([y_folds[j] for j in range(k) if j != i])
    x_test = x_folds[i]
    y_test = y_folds[i]

    scaler = StandardScaler()
    scaler = scaler.fit(x_tr)
    x_tr = scaler.transform(x_tr)
    x_test = scaler.transform(x_test)

    print("sklearn")
    logreg = LogisticRegression(multi_class='multinomial', max_iter=1000)
    logreg.fit(x_tr, y_tr)
    p = logreg.predict_proba(x_test)
    loss = log_loss(y_test, p)
    print(loss.mean())
    losses[0][id_splits[i]] = log_loss(y_test, p)
    accuracies[0][id_splits[i]] = (y_test == np.argmax(p, axis=1))
    preds[0][id_splits[i], :] = p

    print("my")
    model = ANNClassification(units=[20], lambda_=0.3)
    model.fit(x_tr, y_tr)
    p = model.predict(x_test)
    loss = log_loss(y_test, p)
    print(loss.mean())
    losses[1][id_splits[i]] = log_loss(y_test, p)
    accuracies[1][id_splits[i]] = (y_test == np.argmax(p, axis=1))
    preds[1][id_splits[i], :] = p

    models.append(model)

sklearn
0.6422127909468668
my
0.5606179765470954
sklearn
0.6400726031788525
my
0.5505507002489533
sklearn
0.6420233480454398
my
0.5576981705560102
sklearn
0.6506914725278496
my
0.568917115199162


In [16]:
imena = ["MultinomialRegression", "ANN"]
for i in range(n_poskusov):
    print("")
    print(imena[i])
    print(f"Loss {i}: {losses[i].mean()} +/- {losses[i].std() / np.sqrt(len(losses[i]))}")
    print(f"Accuracy {i}: {accuracies[i].mean()} +/- {accuracies[i].std() / np.sqrt(len(accuracies[i]))}")


MultinomialRegression
Loss 0: 0.6437500536747522 +/- 0.004714560560621403
Accuracy 0: 0.76204 +/- 0.001904389867647904

ANN
Loss 1: 0.5594459906378052 +/- 0.0040375631979530785
Accuracy 1: 0.78574 +/- 0.001834953145995832


In [ ]:
m = models[0]


In [56]:
scaler = StandardScaler()
scaler = scaler.fit(train_X)
train_X = scaler.transform(train_X)

test_X = df_test.drop(columns=['id']).to_numpy(dtype=np.float32)
test_X = scaler.transform(test_X)
test_X.shape

(11878, 93)

In [57]:
preds1 = preds
np.random.seed(42)
model = ANNClassification(units=[20], lambda_=0.3)
model.fit(train_X, train_y)
preds = model.predict(test_X)

In [58]:
preds

array([[2.81048030e-04, 2.99572513e-01, 4.04495833e-01, ...,
        1.82667558e-02, 7.62892304e-05, 1.38864712e-04],
       [7.69196775e-04, 2.20415483e-05, 1.24113760e-05, ...,
        1.19734770e-04, 9.98703288e-01, 2.01958193e-04],
       [2.58173076e-04, 6.75001016e-02, 4.94813620e-04, ...,
        2.87216676e-03, 2.39109926e-03, 1.64181154e-03],
       ...,
       [1.93783545e-05, 1.16840228e-03, 2.63070622e-04, ...,
        1.05877926e-05, 1.20521221e-04, 5.02678777e-06],
       [1.96420836e-02, 2.76328326e-04, 1.27300805e-04, ...,
        1.03470531e-03, 7.23753978e-04, 5.65596751e-01],
       [3.06228163e-06, 7.48346767e-01, 2.33591552e-01, ...,
        1.46614201e-04, 5.43289141e-06, 1.17788065e-05]])

In [19]:
# preds = m.predict(test_X)

# save using pickle
import pickle

with open('preds.pkl', 'wb') as f:
    pickle.dump(preds, f)

In [20]:
ids = np.arange(1, preds.shape[0] + 1)

df = pd.DataFrame(preds, columns=['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
df['id'] = ids
df = df[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']]

df.to_csv('final.txt', index=False)

In [59]:
preds.sum(axis=0)

array([ 353.62744533, 3128.81515638, 1526.13458757,  509.15399007,
        535.26851987, 2716.22034738,  529.1706147 , 1659.44732256,
        920.16201615])

In [39]:
ddd = pd.read_csv('final.txt')

In [40]:
ddd = ddd.drop(columns=['id']).to_numpy()

In [70]:
ddd.mean(axis=0)

array([0.02977163, 0.26341262, 0.12848414, 0.0428653 , 0.04506386,
       0.22867657, 0.04455048, 0.13970764, 0.07746776])

In [89]:
pp = model.predict(train_X)

np.min(np.min(ddd, axis=1))

1.0761679969514436e-09

In [75]:
np.std(pp, axis=1)

array([0.30281876, 0.17555113, 0.22487851, ..., 0.31122848, 0.28821334,
       0.24330885])